In [2]:
from sklearn.neural_network import MLPClassifier as mlpc

import numpy as np
import matplotlib.pyplot as plt

import mltools as ml

In [7]:
X = np.genfromtxt('data/X_train.txt', delimiter=',')
Y = np.genfromtxt('data/Y_train.txt', delimiter=',')
np.random.seed(0)

X,Y = ml.shuffleData(X,Y)
X = X[:,:41]

In [8]:
Xtr, Xva, Ytr, Yva = ml.splitData(X,Y,0.7)

In [ ]:
learner = mlpc()#no weights first.
learner.fit(X,Y) #the training

In [12]:
arr = learner.predict(Xva)